In [1]:
%%file word_count.py

from mrjob.job import MRJob
from mrjob.step import MRStep
from itertools import tee
import re
import sys

WORD_KEY = re.compile(r"[\w']+")

# New class for MRJob Word Probability
class MapReduce(MRJob):
    
    def steps(self):
        return [
            MRStep(mapper=self.input_file),
            MRStep(mapper=self.set_bigrams,
                   reducer=self.count_bigrams),
            MRStep(reducer=self.solution)
        ]
    
    # Step 1 - Edits the file's lines to make them more "readable"
    def input_file(self, _, line):
        if(line[0] != '"'):
            yield (None, line[line.find(","):].lower())
    
    # Step 2 - Pull words from the string and make a bigram for every word
    def set_bigrams(self, _, line):
        prev = ""
        # Find all the words
        for word in WORD_KEY.findall(line):
            if(prev != ""):
                yield ((prev, word), 1)
            prev = word
    
    # Step 2 - Combine all like bigrams
    def count_bigrams(self, word, counts):
        first_word, second_word = word
        yield first_word, (sum(counts), second_word)
        
    # Function used to sort by probability
    def mostUsed(self, x):
        num, word = x
        return num
    
    # Step 3 - Calculate percentages
    def solution(self, word, pairs):
        total = 0
        
        # Tee off the iterator so we can have 3 total runs through the data
        pairs, secondPairs = tee(pairs)
        pairs, secondPairs = tee(pairs)
        
        # First, calculate the total number of occurences of each bigram
        for pair in pairs:
            tempCount, _ = pair
            total = total + tempCount
        
        # Second, create and print the probability list
        probabilityList = sorted(secondPairs, key=self.mostUsed, reverse = True)
        for tempPair in probabilityList:
            word_count, word_key = tempPair
            yield (word, word_key), ((float(word_count) / total), word_count)
        
        # Third, if the first word is "my", print the most used pairs
        if (word == "my"):
            for i in range(10):
                if i == len(probabilityList): 
                    break
                word_count, word_key = probabilityList[i]
                yield 'Most Used - ' + str(i+1), ((word, word_key), word_count / total, word_count)
        
        # Third, if the first word is "my", print the most used pairs
        if (word == "my"):
            for i in range(10):
                if i == len(probabilityList): 
                    break
                word_count, word_key = probabilityList[i]
                yield 'Most Used - ' + str(i+1), ((word, word_key), word_count / total, word_count)
        
        

# Run the program
if __name__ == '__main__':
    MapReduce.run()

Overwriting word_count.py


In [2]:
!pip install mrjob

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!python word_count.py -r local shortjokes.csv --output-dir=word_count_out --no-output

Using configs in /home/nbuser/.mrjob.conf
No configs specified for local runner
Creating temp directory /tmp/word_count.nbuser.20190520.052949.743500
Running step 1 of 3...
Running step 2 of 3...
Running step 3 of 3...
job output is in word_count_out
Removing temp directory /tmp/word_count.nbuser.20190520.052949.743500...


In [5]:
!python word_count.py -r emr s3://cs351-mapreduce/input/shortjokes.csv \
--output-dir=s3://cs351-mapreduce/word_count_out \
--no-output

Using configs in /home/nbuser/.mrjob.conf
Auto-created temp S3 bucket mrjob-d74222449ba74b80
Using s3://mrjob-d74222449ba74b80/tmp/ as our temp dir on S3
Creating temp directory /tmp/word_count.nbuser.20190520.053323.224738
writing master bootstrap script to /tmp/word_count.nbuser.20190520.053323.224738/b.sh
uploading working dir files to s3://mrjob-d74222449ba74b80/tmp/word_count.nbuser.20190520.053323.224738/files/wd...
Copying other local files to s3://mrjob-d74222449ba74b80/tmp/word_count.nbuser.20190520.053323.224738/files/
Created new cluster j-3IVPRQQ73DZ94
Added EMR tags to cluster j-3IVPRQQ73DZ94: __mrjob_label=word_count, __mrjob_owner=nbuser, __mrjob_version=0.6.8
Waiting for Step 1 of 3 (s-2JYTNQPYC6M3J) to complete...
  PENDING (cluster is STARTING)
  PENDING (cluster is STARTING)
  PENDING (cluster is STARTING)
  PENDING (cluster is STARTING: Configuring cluster software)
  PENDING (cluster is BOOTSTRAPPING: Running bootstrap actions)
  PENDING (cluster is BOOTSTRAPPING: 

In [4]:
%%file ~/.mrjob.conf

# http://mrjob.readthedocs.io/en/stable/guides/emr-opts.html

runners:
  emr:
    aws_access_key_id: 
    aws_secret_access_key: 
    ec2_key_pair: 
    ec2_key_pair_file: 
    region: us-east-2 # http://docs.aws.amazon.com/general/latest/gr/rande.html
    master_instance_type: m5.xlarge # https://aws.amazon.com/emr/pricing/
    instance_type: m5.xlarge
    num_core_instances: 1
    ssh_tunnel: true

Overwriting /home/nbuser/.mrjob.conf
